In [65]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary


import time


In [66]:
def filterWords(words):
    newWords = []
    for word in words:
        word = word.lower()
        word = word.replace("!", "")
        word = word.replace("?", "")
        word = word.replace(":", "")
        word = word.replace(";", "")
        word = word.replace(".", "")
        word = word.replace(",", "")
        word = word.replace('\'', "")
        word = word.replace('\"', "")
        #words below are ones where we want to connect together to ensure these words are joined
        word = word.replace("trumps", "trump")
        word = word.replace("russian", "russia")
        word = word.replace("russians", "russia")
        word = word.replace("dems", "democrat")
        word = word.replace("democrats", "democrat")
        word = word.replace("republicans", "republican")
        word = word.replace("libs", "liberal")
        word = word.replace("liberals", "liberal")
        word = word.replace("conservatives", "conservative")
        word = word.replace("taxes", "tax")
        word = word.replace("socialists", "socialist")
        word = word.replace("nationalists", "nationalist")
        word = word.replace("globalists", "globalist")
        word = word.replace("corrupted", "corrupt")
        newWords.append(word)
    return newWords

In [131]:
# Scrape Huffington (limited history = 15 pages --> 260 articles)
# Set up arrays 
article_title = []
article_URL = []
interaction_count = []

def scrapeTest(url):
    articlesDF = []
    wordsDF = []
    html_doc = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(html_doc.text, 'html.parser')
   
    articleList = soup.find('div', {'class': 'a-page__column--center'})
    articleList = articleList.find("div", {"class": "zone__content"})
    articles = articleList.findAll("div", {"class": "card"})
   
    
    for article in articles:
            linkHTML = article.find("a", {"class": "card__link yr-card-headline"})
            link = linkHTML['href']
            linkURL = "https://www.huffingtonpost.com" + link
            articleTitle = article.find('div', {'class': 'card__headline__text'}).text
            wordsInTitle = articleTitle.split()
            wordsInTitle = filterWords(wordsInTitle)
            article_title.append(re.sub("\n", " ", articleTitle))
            article_URL.append(linkURL)
            
for x in range (2, 15):
    url = 'https://www.huffingtonpost.com/section/politics?page=' + str(x)
    scrapeTest(url)



In [148]:
# Get time stamp and comment count of each article 
time_stamp = []
comment_count = []


def getTimeComments(url): 
    single_req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(single_req.text, 'html.parser')
    time = soup.find('span', {'class' : "timestamp__date--published"}).text
    time_stamp.append(time)
    
#     count = soup.find('div', {'class' : "js-sharebar"})
#     countFind = count.find('span', {'class' : 'share-bar__social-counts__number'}).text
#     comment_count.append((countFind))

for link in article_URL: 
    getTimeComments(link)
     

In [153]:
# Get interaction comments

def convertNum(number):
    if "k" in str(number): 
        return float(re.sub('k', "", number)) * 1000
    else: 
        return float(number)
    

def getComment(url):
    html_doc = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    find = soup.find('div', {'class' : "js-sharebar"})
    try: 
        test = find.find('span', {'class' : 'share-bar__social-counts__number'}).text
        return comment_count.append(convertNum(test))
    except: return comment_count.append(-1) 

for link in article_URL: 
    getComment(link)


In [ ]:
def filterWords(words):
    newWords = []
    for word in words:
        word = word.lower()
        word = word.replace("!", "")
        word = word.replace("?", "")
        word = word.replace(":", "")
        word = word.replace(";", "")
        word = word.replace(".", "")
        word = word.replace(",", "")
        word = word.replace('\'', "")
        word = word.replace('\"', "")
        #words below are ones where we want to connect together to ensure these words are joined
        word = word.replace("trumps", "trump")
        word = word.replace("russian", "russia")
        word = word.replace("russians", "russia")
        word = word.replace("dems", "democrat")
        word = word.replace("democrats", "democrat")
        word = word.replace("republicans", "republican")
        word = word.replace("libs", "liberal")
        word = word.replace("liberals", "liberal")
        word = word.replace("conservatives", "conservative")
        word = word.replace("taxes", "tax")
        word = word.replace("socialists", "socialist")
        word = word.replace("nationalists", "nationalist")
        word = word.replace("globalists", "globalist")
        word = word.replace("corrupted", "corrupt")
        newWords.append(word)
    return newWords

In [207]:
# Set up dataframe 
d = {"title" : filterWords(article_title), "url" : article_URL, 'interaction_count' : comment_count}
articleDF = pd.DataFrame(data=d)
huff_df = articleDF.query('interaction_count>0')
huff_df

,title,url,interaction_count
1,jon voight appears game to be trump chief of ...,https://www.huffingtonpost.com/entry/jon-voght...,510.0
2,accused russia agent maria butina reaches ple...,https://www.huffingtonpost.com/entry/maria-but...,2000.0
3,trump businesses have charged $11 million to ...,https://www.huffingtonpost.com/entry/trump-bus...,1300.0
4,kevin mccarthy says democrat shouldnt spend a...,https://www.huffingtonpost.com/entry/kevin-mcc...,5200.0
5,fbi announces arrests of 2 people allegedly p...,https://www.huffingtonpost.com/entry/toledo-oh...,2900.0
7,protesters arrested outside nancy pelosis off...,https://www.huffingtonpost.com/entry/protester...,240.0
8,supreme court justices wont hear states appea...,https://www.huffingtonpost.com/entry/supreme-c...,260.0
11,dnc mandates staff neutrality in presidential...,https://www.huffingtonpost.com/entry/dnc-tom-p...,540.0
14,trump eyes four new candidates for chief of s...,https://www.huffingtonpost.com/entry/trump-loo...,410.0
15,french government slams trump stop interferin...,https://www.huffingtonpost.com/entry/france-do...,5400.0


In [223]:
# Summarizing data 
wordCount = huff_df['title'].str.split(expand=True).stack().value_counts()
wordCount.index.tolist()
wordCountDF = pd.DataFrame(data={"word" : wordCount.index.tolist(), "articleAppearanceCount" : wordCount.values.tolist()})
fillerWords = ["that", "to", "the", "in", "for", "on", "a", "with", "is", "as", "over", "be", "he", "and", 'of', 'at', 'his', 'about']
wordCountDF = wordCountDF[~wordCountDF.word.isin(fillerWords)]
wordCountDF.to_csv('huffingtonpostWordCounts.csv', index = False)

In [226]:
# Get average comment count per unique word 